In [ ]:
pip install hyperopt

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 

from hyperopt import hp, fmin, atpe, Trials

from hyperopt.pyll.base import scope

from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline

from functools import partial

from sklearn import model_selection

In [2]:
mpl.rcParams['figure.dpi'] = 200
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False

In [3]:
train = pd.read_csv('/Users/lucywilliams/Downloads/tabular-playground-series-aug-2021/train.csv')
test = pd.read_csv('/Users/lucywilliams/Downloads/tabular-playground-series-aug-2021/test.csv')
sample_submission = pd.read_csv('/Users/lucywilliams/Downloads/tabular-playground-series-aug-2021/sample_submission.csv')

In [4]:
train.shape

(250000, 102)

In [5]:
train['kfold']= -1
train = train.sample(frac=1).reset_index(drop=True)
train.loc[:, "bins"] = pd.cut(
 train["loss"], bins=19, labels=False
 )
kf = model_selection.StratifiedKFold(n_splits=10)
for f, (t_, v_) in enumerate(kf.split(X=train, y=train.bins.values)):
    train.loc[v_, 'kfold'] = f
train = train.drop("bins", axis=1)

In [6]:
X=train.iloc[:,1:].copy()
X.drop('loss',axis=1,inplace=True)

In [7]:
target =['loss', 'kfold']
X=X
y=train[target]

In [8]:
def optimize(params,x,y):
    model= XGBRegressor(n_jobs=2,nthread=2,random_state=0,**params)
    RMSE = []
    for i in range(13,15):
        X_t=x[x.kfold != i]
        X_train=X_t.iloc[:,:-1]
        y_t=y[y.kfold !=i]
        y_train=y_t.iloc[:,0].values
        X_te=x[x.kfold == i]
        X_test=X_te.iloc[:,:-1]
        y_te=y[y.kfold==i]
        y_test=y_te.iloc[:,0].values
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        fold_ras = mean_squared_error(y_test, preds, squared=False)
        RMSE.append(fold_ras)
    print(RMSE)
    return (np.mean(RMSE))

In [9]:
param_space = {
 "learning_rate": hp.uniform("learning_rate", 0.008, 0.06),
 "max_depth": scope.int(hp.quniform("max_depth",10 , 11, 1)),
 "n_estimators": scope.int(hp.quniform("n_estimators", 1650, 1750, 1)),
    
 "tree_method": hp.choice("tree_method", ["gpu_hist"]),
 "booster": hp.choice("booster", ["gbtree"]),
 "objective": hp.choice("objective", ["reg:squarederror"]),
 "eval_metric": hp.choice("eval_metric", ["rmse"]),
 "predictor": hp.choice("predictor", ["gpu_predictor"]),
 "gpu_id": hp.choice("gpu_id", [0]),

 "subsample": hp.uniform("subsample", 0.7, 0.98),
    "min_child_weight": hp.uniform("min_child_weight",15, 40),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.75, 0.96),
    "reg_alpha": hp.uniform("reg_alpha", 44, 90),
    "reg_lambda": hp.uniform("reg_lambda", 50, 90),
    "gamma": hp.uniform("gamma", 0.5, 10),
    "max_delta_step": hp.uniform("max_delta_step", 0.1, 3),
    "colsample_bylevel": hp.uniform("colsample_bylevel", 0.7, 0.96),
    "colsample_bynode": hp.uniform("colsample_bynode", 0.6, 0.96)
 }
optimization_function = partial(
 optimize,
x=X, y=y
 )   
trials = Trials() 
hopt = fmin( 
 fn=optimization_function,
 space=param_space,
 algo=atpe.suggest,
 max_evals=20,
 trials=trials
 )
print(hopt)

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]


OSError: dlopen(/Users/lucywilliams/opt/anaconda3/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/lucywilliams/opt/anaconda3/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found